# Making observation plan (date)

## Install & Import modules

In [1]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, astroquery, requests, astropy, astropy.units, certifi, version_information".split(', ')
for package in packages:
    if not importlib.util.find_spec(package):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])
        print(f'{package} is being installed')
    else:
        print(f'{package} is already installed')

import pandas as pd
import json
import requests

from astroquery.mpc import MPC
from astropy.coordinates import SkyCoord
from astropy import units as u

import os

## Set date & condition

In [2]:
obs_date_utc = '2023-09-12'     # YYYY-MM-DD

mpc_code = 'P64'
elev_min = 30
time_min = 15
vmag_min = 6
vmag_max = 15
max_output = 50
sort = 'trans'      # Transit Time

## Bring & Save data, Save xml doc

In [3]:
print(f'Date : {obs_date_utc}')

for obs_time_utc_hour in range(11, 21):     # KST 밤 시간대(20:00 ~ 5:00)
    obs_time_utc = f'{str(obs_time_utc_hour).zfill(2)}:00:00'       # hh:mm:ss
    print(f'\tTime : {obs_time_utc}')
    
    data_set = []

    url = f"https://ssd-api.jpl.nasa.gov/sbwobs.api?sb-kind=a&mpc-code={mpc_code}&obs-time={obs_date_utc}T{obs_time_utc}&elev-min={elev_min}&time-min={time_min}&vmag-max={vmag_max}&vmag-min={vmag_min}&fmt-ra-dec=true&mag-required=true&output-sort={sort}&maxoutput={max_output}"
    res = requests.get(url)
    data = json.loads(res.text)

    for data_line in data['data']:
        TT_hour, TT_min = data_line[3].replace('*','').split(':')
        TransTime_tmp = int(TT_hour) + int(TT_min)/60
    
        ObsTime_tmp = int(obs_time_utc[:2]) + int(obs_time_utc[3:5])/60
    
        if ObsTime_tmp - 0.5 <= TransTime_tmp and TransTime_tmp < ObsTime_tmp + 0.5:
            name = data_line[1].split('(')[1][:-1]
            try:
                result = MPC.get_ephemeris(name, location = mpc_code, start = obs_date_utc, step = '1h')[round(TransTime_tmp)]
            
                c = SkyCoord(ra = result[1]*u.degree, dec = result[2]*u.degree)
                Ra,Dec = c.to_string('hmsdms').split(' ')
                Ra = Ra.replace('h',':').replace('m',':').replace('s','')
                Dec = Dec.replace('d',':').replace('m',':').replace('s','').replace('+','')
            
                Vmag = result[7]
                Azi = result[10]
                Alt = result[11]

                if int(Alt) >= elev_min:
                    data_set.append([name, Ra, Dec, Vmag, Azi, Alt, f'{TT_hour}:{TT_min}'])
            except:
                continue

    ObsPlan = [[30,'r',1,10],[60,'r',1,10],[90,'r',1,10],[30,'r',2,10],[60,'r',2,10],[90,'r',2,10],]   # ExpTime, Filter, Bin, TotalExpCnt
    Sequence_set = []

    for asteroid_data in data_set:
        Targetname = asteroid_data[0].replace(' ','') + '_' + obs_date_utc.replace('-','')[2:] + '_' + obs_time_utc.replace(':','')[:2]    # 제목 형식: Name_yymmdd_hh(UTC)
  
        RAHours, RAMinutes, RASeconds = asteroid_data[1].split(':');RASeconds = RASeconds[:2]   # hh, mm, ss
        DecDegrees, DecMinutes, DecSeconds = asteroid_data[2].split(':');DecSeconds = DecSeconds[:2]    # hh, mm, ss

        pre = """<?xml version="1.0" encoding="utf-8"?>\n"""
        CSlist = f"""<CaptureSequenceList xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" TargetName="{Targetname}" Mode="ROTATE" RAHours="{RAHours}" RAMinutes="{RAMinutes}" RASeconds="{RASeconds}" DecDegrees="{DecDegrees}" DecMinutes="{DecMinutes}" DecSeconds="{DecSeconds}" Rotation="0" Delay="0" SlewToTarget="true" AutoFocusOnStart="true" CenterTarget="true" RotateTarget="false" StartGuiding="true" AutoFocusOnFilterChange="false" AutoFocusAfterSetTime="false" AutoFocusSetTime="30" AutoFocusAfterSetExposures="false" AutoFocusSetExposures="10" AutoFocusAfterTemperatureChange="false" AutoFocusAfterTemperatureChangeAmount="5" AutoFocusAfterHFRChange="false" AutoFocusAfterHFRChangeAmount="10">\n"""
        xmlDoc = pre + CSlist   # xml 문서

        for Obs in ObsPlan:   # 관측 계획(알짜 요소)
            CapSeq = f"""  <CaptureSequence>
    <Enabled>true</Enabled>
    <ExposureTime>{Obs[0]}</ExposureTime>
    <ImageType>LIGHT</ImageType>
    <FilterType>
      <Name>{Obs[1]}</Name>
      <FocusOffset>0</FocusOffset>
      <Position>3</Position>
      <AutoFocusExposureTime>-1</AutoFocusExposureTime>
      <AutoFocusFilter>false</AutoFocusFilter>
      <FlatWizardFilterSettings>
        <FlatWizardMode>DYNAMICEXPOSURE</FlatWizardMode>
        <HistogramMeanTarget>0.5</HistogramMeanTarget>
        <HistogramTolerance>0.1</HistogramTolerance>
        <MaxFlatExposureTime>30</MaxFlatExposureTime>
        <MinFlatExposureTime>0.01</MinFlatExposureTime>
        <StepSize>0.1</StepSize>
        <MaxAbsoluteFlatDeviceBrightness>1</MaxAbsoluteFlatDeviceBrightness>
        <MinAbsoluteFlatDeviceBrightness>0</MinAbsoluteFlatDeviceBrightness>
        <FlatDeviceAbsoluteStepSize>1</FlatDeviceAbsoluteStepSize>
      </FlatWizardFilterSettings>
      <AutoFocusBinning>
        <X>1</X>
        <Y>1</Y>
      </AutoFocusBinning>
      <AutoFocusGain>-1</AutoFocusGain>
      <AutoFocusOffset>-1</AutoFocusOffset>
    </FilterType>
    <Binning>
      <X>{Obs[2]}</X>
      <Y>{Obs[2]}</Y>
    </Binning>
    <Gain>-1</Gain>
    <Offset>-1</Offset>
    <TotalExposureCount>{Obs[3]}</TotalExposureCount>
    <ProgressExposureCount>0</ProgressExposureCount>
    <Dither>true</Dither>
    <DitherAmount>1</DitherAmount>
  </CaptureSequence>\n"""
            xmlDoc += CapSeq

        end = f"""  <Coordinates>
    <RA>{int(RAHours) + int(RAMinutes)/60 + int(RASeconds)/3600}</RA>
    <Dec>{int(DecDegrees) + int(DecMinutes)/60 + int(DecMinutes)/3600}</Dec>
    <Epoch>J2000</Epoch>
  </Coordinates>
  <NegativeDec>false</NegativeDec>
</CaptureSequenceList>\n"""
        xmlDoc += end
  
        Sequence_set.append([Targetname, xmlDoc])   # 제목, xml 문서 내용

    path = f"D:\Downloads\\{obs_date_utc.replace('-','')[2:]}\\{obs_time_utc[:2]}\\FileName.xml"     # FileName 자리에 xml 문서 이름
    try:
        if not os.path.exists(path[:-13]):
            os.makedirs(path[:-13])     # File 만들기
    except OSError:
        continue

    for Sequence in Sequence_set:
        print(f'\t\t{Sequence[0]}')
        file = open(path.replace('FileName',Sequence[0]), 'w+')
        file.write(Sequence[1])
        file.close

Date : 2023-09-12
	Time : 11:00:00
		A896JB_230912_11
		A894VC_230912_11
	Time : 12:00:00
		A922MA_230912_12
		A913QC_230912_12
		A918LA_230912_12
		1929UA_230912_12
		A919NC_230912_12
		A895KA_230912_12
		1933NA_230912_12
		A918EN_230912_12
		A916BC_230912_12
		A906TC_230912_12
	Time : 13:00:00
		A895FB_230912_13
		1939PJ_230912_13
		A898PA_230912_13
		A894WC_230912_13
		A892UA_230912_13
		A918PJ_230912_13
		A904PE_230912_13
		A908QH_230912_13
		A907FC_230912_13
		A909BP_230912_13
		A913YF_230912_13
	Time : 14:00:00
		1926RG_230912_14
		1931TG_230912_14
		A905YD_230912_14
	Time : 15:00:00
	Time : 16:00:00
	Time : 17:00:00
	Time : 18:00:00
	Time : 19:00:00
	Time : 20:00:00
